# Домашняя работа 4. Гриневская Катя, БКЛ182

Я бы очень хотела использовать и проверку моего кода на соответствие pep-8, и замерить время работы, но, к сожалению, pep-8 и %%time не работают одновременно. Я проверила свой код на pep-8 в другой тетрадке, её я тоже прикрепила в репозиторий, в которой я на малюсеньком тексте провела эту же работу. В этой тетрадке я не буду устанавливать pip, чтобы посчитать время работы моего кода.

In [ ]:
# ! pip install pycodestyle

In [ ]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

### Задание 2. Обработать книгу с помощью mystem:

- распарсить с помощью mystem
- замерить время работы
- сохранить результат в json

In [1]:
# устанавливаем mystem
! pip install pymystem3
from pymystem3 import Mystem
m = Mystem()

In [2]:
#открываем книгу
with open('андерсон_яйцо.txt', encoding='utf-8') as fh:
    text = fh.read()

In [3]:
# ипортируем json 
import json

In [5]:
# считаем время и записываем результат в файл
%time ana = m.analyze(text)
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(ana, f, ensure_ascii=False, indent=4)

Wall time: 7min 31s


## Задание 3. Обработать книгу через pymorphy

- токенизировать текст с помощью nltk
- разобрать слова с помощью pymorphy
- замерить время работы
- сохраните результат (хотя бы лемма + часть речи) в json

In [7]:
# устанавливаем pymorphy
! pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [8]:
# токенизируем текст
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
%%time
# создаём список, который потом отправим в файл json
f2_list = []
# создаём переменную для кол-ва слов
tok_num = 0
for token in tokens:
    # очищаем от знаков препинания
    if token not in """!@#$%^&*()_+'?/.<>,|\"~№=-:;""":
        tok_num += 1
        # разбираем токены с помощью pymorphy
        anal = morph.parse(token)
        token_dict = {}
        token_dict['Слово'] = anal[0].word
        token_dict['Лемма'] = anal[0].normal_form
        token_dict['Часть речи'] = anal[0].tag.POS
        f2_list.append(token_dict)

Wall time: 1.11 s


In [10]:
# вносим всё в новый файл
with open('data_2.json', 'w', encoding='utf-8') as fi:
    json.dump(f2_list, fi, ensure_ascii=False, indent=4)

###### А ещё у меня проблемы с принтом, и с обычным, и с pprint, поэтому у меня не получится сделать красивый вывод, а только самый обыкновенный (просто вывести переменную)

## Задание 4. Ответить на вопросы:

- Какую долю слов составляет каждая часть речи? (Например, для глагола - это количество глаголов, деленное на общее число слов в тексте)
- Найдите топ-20 (по частотности) глаголов и наречий

In [11]:
from collections import Counter
# список для всех частей речи
gr_list = []
# словарь с парами (часть речи; частота в тексте)
gr_freq = {}
# заполняем список частями речи, встр. в тексте
for token in tokens:
    if token not in """!@#$%^&*()_+'?/.<>,|\"~№=-:;""":
        anal = morph.parse(token)
        gr_list.append(anal[0].tag.POS)
gr_dict = Counter(gr_list)
# заполняем словарь с парами (часть речи; частота)
for key in gr_dict.keys():
    numb = int(gr_dict[key])
    freq = str((numb/tok_num)*100) + '%'
    gr_freq[key] = freq
# повторяюсь, вывести могу только так
gr_freq

{'NPRO': '8.462958080674927%',
 'ADJS': '0.9754811494858951%',
 'CONJ': '10.150276825731611%',
 'PREP': '12.47034010018455%',
 'ADJF': '10.176641181123122%',
 'NOUN': '27.99894542578434%',
 'VERB': '13.735829158977065%',
 'NUMR': '0.7909306617453202%',
 'PRTF': '1.1336672818349591%',
 'ADVB': '6.432902715528606%',
 'INFN': '2.7946216715001317%',
 'PRCL': '2.610071183759557%',
 'GRND': '0.8963880833113631%',
 None: '0.5009227524387029%',
 'PRED': '0.3163722646981281%',
 'PRTS': '0.3691009754811495%',
 'COMP': '0.15818613234906406%',
 'INTJ': '0.02636435539151068%'}

###### Второй пункт отдельно выведу для глаголов:

In [12]:
# список для глаголов
v_list = []
# список для наречий
a_list = []
for token in tokens:
    if token not in """!@#$%^&*()_+'?/.<>,|\"~№=-:;""":
        anal = morph.parse(token)
        if (anal[0].tag.POS == 'VERB')or(anal[0].tag.POS == 'INFN'):
            v_list.append(anal[0].normal_form)
        if anal[0].tag.POS == 'ADVB':
            a_list.append(anal[0].normal_form)
Counter(v_list).most_common(20)

[('быть', 65),
 ('стать', 11),
 ('знать', 8),
 ('начать', 8),
 ('поставить', 8),
 ('мочь', 7),
 ('говорить', 7),
 ('сказать', 6),
 ('приходить', 6),
 ('иметь', 5),
 ('ждать', 5),
 ('думать', 5),
 ('упасть', 5),
 ('сделать', 5),
 ('читать', 4),
 ('идти', 4),
 ('видеть', 4),
 ('хотеть', 4),
 ('находиться', 4),
 ('бывать', 4)]

###### И отдельно для наречий:

In [13]:
Counter(a_list).most_common(20)

[('затем', 13),
 ('снова', 9),
 ('несколько', 8),
 ('где', 7),
 ('уже', 7),
 ('много', 6),
 ('прежде', 4),
 ('всего', 4),
 ('более', 4),
 ('там', 4),
 ('теперь', 4),
 ('обратно', 4),
 ('потом', 3),
 ('несомненно', 3),
 ('возле', 3),
 ('иногда', 3),
 ('почему', 3),
 ('редко', 3),
 ('вечером', 3),
 ('здесь', 3)]

## Задание 5. Посмотрите документацию для nltk н-грамм (nltk.bigrams, например), попробуйте составить топ-25 биграмм и триграмм для вашего текста в лемматизированном виде (только леммы, без знаков препинания). Почему получаются именно такие?

##### отдельно для биграмм:

In [14]:
# пустой список для лемматиз. текста
lemmas_text = []
for token in tokens:
    if token not in """!@#$%^&*()_+'?/.<>,|\"~№=-:;""":
        anal = morph.parse(token)
        lemmas_text.append(anal[0].normal_form)
# список биграм лемматиз. текста
lemmas_bi = list(nltk.bigrams(lemmas_text))
Counter(lemmas_bi).most_common(25)

[(('что', 'он'), 10),
 (('он', 'быть'), 8),
 (('о', 'тот'), 8),
 (('с', 'матерь'), 8),
 (('в', 'город'), 7),
 (('тот', 'что'), 7),
 (('яйцо', 'и'), 7),
 (('джо', 'кейн'), 7),
 (('в', 'это'), 6),
 (('в', 'тот'), 6),
 (('а', 'затем'), 6),
 (('он', 'в'), 6),
 (('на', 'прилавок'), 6),
 (('и', 'с'), 5),
 (('у', 'он'), 5),
 (('на', 'свет'), 5),
 (('и', 'я'), 5),
 (('что', 'я'), 5),
 (('поставить', 'яйцо'), 5),
 (('уверенный', 'что'), 4),
 (('что', 'у'), 4),
 (('на', 'ферма'), 4),
 (('город', 'бидуэла'), 4),
 (('прежде', 'всего'), 4),
 (('что', 'в'), 4)]

##### и отдельно для триграмм:

In [17]:
lemmas_tri = list(nltk.trigrams(lemmas_text))
Counter(lemmas_tri).most_common(25)

[(('мы', 'с', 'матерь'), 3),
 (('быть', 'уверенный', 'что'), 3),
 (('что', 'он', 'хотеть'), 3),
 (('у', 'он', 'быть'), 3),
 (('поставить', 'яйцо', 'стоймя'), 3),
 (('город', 'бидуэла', 'штат'), 2),
 (('бидуэла', 'штат', 'огайо'), 2),
 (('ферма', 'в', 'город'), 2),
 (('в', 'бар', 'бена'), 2),
 (('бар', 'бена', 'хеда'), 2),
 (('в', 'десять', 'часы'), 2),
 (('в', 'тот', 'время'), 2),
 (('о', 'тот', 'чтобы'), 2),
 (('в', 'это', 'мир'), 2),
 (('читать', 'книга', 'и'), 2),
 (('о', 'тот', 'как'), 2),
 (('появление', 'на', 'свет'), 2),
 (('о', 'тот', 'что'), 2),
 (('с', 'самый', 'начало'), 2),
 (('в', 'пот', 'лицо'), 2),
 (('к', 'свой', 'создатель'), 2),
 (('отец', 'с', 'матерь'), 2),
 (('в', 'город', 'бидуэла'), 2),
 (('заняться', 'ресторанный', 'дело'), 2),
 (('а', 'затем', 'в'), 2)]